# Page Analysis

In [1]:
import pandas as pd
import dask.dataframe as ddf
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import helper_fns
%matplotlib inline

# Initial data exploration

In [2]:
df = ddf.read_csv('page_data_v2.csv')
df.head()

,id,sess_id,attn_10,attn_20,attn_30,attn_40,attn_50,attn_60,attn_70,attn_80,...,domain,domain_id,ajax_reqs,num_events,num_forms,avg_form_fill_s,avg_time2click_ms,conversion_count,conversion_value,conv_rate
0,21995205,3077876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,4,4,0,NaN,NaN,0,0,NaN
1,21994704,3077742,49.0,17.0,0.0,17.0,0.0,17.0,17.0,0.0,...,www.google.com,9,6,4,0,NaN,NaN,0,0,NaN
2,21994274,3077634,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,www.google.com,9,6,5,0,NaN,NaN,0,0,NaN
3,21994351,3077664,73.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,www.google.com,9,6,5,0,NaN,NaN,0,0,NaN
4,21996192,3078118,39.0,81.0,81.0,81.0,81.0,100.0,19.0,19.0,...,www.google.com,9,5,5,0,NaN,NaN,0,0,NaN


In [3]:
df.columns

Index(['id', 'sess_id', 'attn_10', 'attn_20', 'attn_30', 'attn_40', 'attn_50',
       'attn_60', 'attn_70', 'attn_80', 'attn_90', 'attn_100', 'clicks',
       'engaged_seconds', 'max_scroll', 'url_id', 'category', 'category_id',
       'domain', 'domain_id', 'ajax_reqs', 'num_events', 'num_forms',
       'avg_form_fill_s', 'avg_time2click_ms', 'conversion_count',
       'conversion_value', 'conv_rate'],
      dtype='object')

In [4]:
df.compute().shape

(2400000, 28)

In [5]:
# exploring some of the categorical features
print('{} unique referral categories: {}'.format(len(df.category.compute().unique()), df.category.compute().unique()))
print('{} unique domains: {}'.format(len(df.domain.compute().unique()), df.domain.compute().unique()))

1 unique referral categories: [nan]
35 unique domains: [nan 'www.google.com' 'www.bing.com' 'www.yahoo.com' 'www.live.com'
 'instagram.com' 'www.retailmenot.com' 'facebook.com' 'rdi.criteo.com'
 'pinterest.com' 'www.youtube.com' 'www.ask.com' 'm.youtube.com'
 'snapchat.com' 'www.duckduckgo.com' 'www.baidu.com'
 'www.quinnandmurray.com' 'www.trafficsafe.net' 'portal.opinionlab.com'
 'quantummetric.atlassian.net' 'quinnandmurray.com'
 'www.quartermanfinancial.com' 'qpartnerdev.co1.qualtrics.com' 'localhost'
 'r.search.yahoo.com' 'app.segment.com' 'oo.opinionlab.com'
 'www.vistaprint.com' 'www.aa.com' 'www.aavacations.com'
 'www.vistaprint.ie' 'nebula.kampyle.com' 'quartermanfinancial.com'
 'app.optimizely.com' 'quarterman.quantummetric.com']


# Select columns we want to use

In [6]:
# not choosing rtt_ms b.c. it has too many NaNs, not choosing engaged because it only has 1 value
subset_df = df[['conv_rate','attn_10', 'attn_20', 'attn_30', 'attn_40', 'attn_50', 'attn_60', 'attn_70', 'attn_80', 'attn_90', 'attn_100', 'clicks',
                'engaged_seconds', 'max_scroll', 'domain', 'ajax_reqs', 'num_events', 'num_forms', 'avg_form_fill_s', 'avg_time2click_ms']]
subset_df.head()

,conv_rate,attn_10,attn_20,attn_30,attn_40,attn_50,attn_60,attn_70,attn_80,attn_90,attn_100,clicks,engaged_seconds,max_scroll,domain,ajax_reqs,num_events,num_forms,avg_form_fill_s,avg_time2click_ms
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,4,4,0,NaN,NaN
1,NaN,49.0,17.0,0.0,17.0,0.0,17.0,17.0,0.0,0.0,17.0,0,10,100.0,www.google.com,6,4,0,NaN,NaN
2,NaN,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0,6,100.0,www.google.com,6,5,0,NaN,NaN
3,NaN,73.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,76.0,0,8,100.0,www.google.com,6,5,0,NaN,NaN
4,NaN,39.0,81.0,81.0,81.0,81.0,100.0,19.0,19.0,19.0,19.0,0,10,60.0,www.google.com,5,5,0,NaN,NaN


In [7]:
subset_df.compute().shape

(2400000, 20)

# Initial data cleanup

In [8]:
subset_df['conv_rate'] = subset_df['conv_rate'].fillna(0)
subset_df['avg_form_fill_s'] = subset_df['avg_form_fill_s'].fillna(0)
subset_df['avg_time2click_ms'] = subset_df['avg_time2click_ms'].fillna(0)
subset_df = subset_df.dropna(subset=['domain'])

subset_df.head()

,conv_rate,attn_10,attn_20,attn_30,attn_40,attn_50,attn_60,attn_70,attn_80,attn_90,attn_100,clicks,engaged_seconds,max_scroll,domain,ajax_reqs,num_events,num_forms,avg_form_fill_s,avg_time2click_ms
1,0.0,49.0,17.0,0.0,17.0,0.0,17.0,17.0,0.0,0.0,17.0,0,10,100.0,www.google.com,6,4,0,0.0,0.0
2,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0,6,100.0,www.google.com,6,5,0,0.0,0.0
3,0.0,73.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,76.0,0,8,100.0,www.google.com,6,5,0,0.0,0.0
4,0.0,39.0,81.0,81.0,81.0,81.0,100.0,19.0,19.0,19.0,19.0,0,10,60.0,www.google.com,5,5,0,0.0,0.0
5,0.0,51.0,82.0,82.0,82.0,100.0,100.0,100.0,30.0,18.0,18.0,0,12,100.0,www.google.com,6,7,0,0.0,0.0


In [9]:
subset_df.compute().shape

(2390851, 20)

In [10]:
subset_df.compute().describe()

,conv_rate,attn_10,attn_20,attn_30,attn_40,attn_50,attn_60,attn_70,attn_80,attn_90,attn_100,clicks,engaged_seconds,max_scroll,ajax_reqs,num_events,num_forms,avg_form_fill_s,avg_time2click_ms
count,2.390851e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,1.058626e+06,2.390851e+06,2.390851e+06,1.058626e+06,2.390851e+06,2.390851e+06,2.390851e+06,2.390851e+06,2.390851e+06
mean,7.170889e+03,1.801875e+01,2.093465e+01,2.196413e+01,2.176311e+01,2.100001e+01,1.998152e+01,1.813691e+01,1.623306e+01,1.447966e+01,1.296102e+01,1.379342e+00,6.932678e+00,8.677628e+01,1.729937e+00,4.067979e+00,3.357290e-01,1.507442e+00,8.490993e+03
std,2.897048e+04,3.680860e+01,3.944872e+01,4.038649e+01,4.047161e+01,4.026636e+01,3.942360e+01,3.756423e+01,3.561385e+01,3.373557e+01,3.203704e+01,4.587478e+00,1.598251e+01,2.084579e+01,2.260712e+00,2.972473e+00,6.543852e-01,7.636072e+00,1.133739e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,3.698000e+03
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+02,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,6.167000e+03
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,1.000000e+02,4.000000e+00,5.000000e+00,1.000000e+00,0.000000e+00,1.027400e+04
max,3.309960e+05,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,9.600000e+01,5.940000e+02,1.000000e+02,2.800000e+01,4.000000e+01,3.000000e+00,5.010000e+02,5.887140e+05


# Encode categorical data with OneHot encodings

In [11]:
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'domain')

subset_df = subset_df.drop(columns=['domain'])

subset_df.head()

,conv_rate,attn_10,attn_20,attn_30,attn_40,attn_50,attn_60,attn_70,attn_80,attn_90,...,"(www.google.com,)","(www.live.com,)","(www.quartermanfinancial.com,)","(www.quinnandmurray.com,)","(www.retailmenot.com,)","(www.trafficsafe.net,)","(www.vistaprint.com,)","(www.vistaprint.ie,)","(www.yahoo.com,)","(www.youtube.com,)"
1,0.0,49.0,17.0,0.0,17.0,0.0,17.0,17.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,73.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,39.0,81.0,81.0,81.0,81.0,100.0,19.0,19.0,19.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,51.0,82.0,82.0,82.0,100.0,100.0,100.0,30.0,18.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save dataset for future, more efficient, model development and analysis

In [12]:
subset_df.compute().to_csv('page_data_v2_cleaned.csv', index=False)